In [13]:
import requests

url = 'https://query.wikidata.org/sparql'
query = """
SELECT 
  ?country ?countryLabel ?population ?area ?medianIncome ?age
WHERE {
  ?country wdt:P463 wd:Q458.
  OPTIONAL { ?country wdt:P1082 ?population }
  OPTIONAL { ?country wdt:P2046 ?area }
  OPTIONAL { ?country wdt:P3529 ?medianIncome }
  OPTIONAL { ?country wdt:P571 ?inception. 
    BIND(year(now()) - year(?inception) AS ?age)
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
r = requests.get(url, params = {'format': 'json', 'query': query})
data = r.json()

In [16]:
import pandas as pd
from collections import OrderedDict

countries = []
for item in data['results']['bindings']:
    countries.append(OrderedDict({
        'countryCode': item['country']['value'].split("/")[-1],
        'country': item['countryLabel']['value'],
        'population': item['population']['value'],
        'area': item['area']['value'] 
            if 'area' in item else None,
        'medianIncome': item['medianIncome']['value'] 
            if 'medianIncome' in item else None,
        'age': item['age']['value'] 
            if 'age' in item else None}))

df = pd.DataFrame(countries)
df.set_index('country', inplace=True)
df = df.astype({'countryCode':str ,'population': float, 'area': float, 'medianIncome': float, 'age': float})
df.head()

,countryCode,population,area,medianIncome,age
country,,,,,
France,Q142,66628000.0,643801.0,30364.0,NaN
Germany,Q183,83149300.0,357400.0,33333.0,NaN
Kingdom of the Netherlands,Q29999,17100715.0,42201.0,NaN,205.0
Ireland,Q27,4761865.0,70273.0,25969.0,83.0
Belgium,Q31,11431406.0,30528.0,31112.0,190.0
